# Бенчмарк для задачи OneTwoTrip Contest

https://boosters.pro/championship/onetwotrip_challenge/overview
    
    
для студентов ВМК МГУ

2019, Александр Дьяконов www.dyakonov.org/ag/

In [1]:
# подгружаем все нужные пакеты
import pandas as pd
import numpy as np
# для встроенных картинок
%pylab inline
# отключить предупреждения
import warnings
warnings.filterwarnings('ignore')
# прогресс-бар
from tqdm import tqdm, tqdm_notebook

Populating the interactive namespace from numpy and matplotlib


In [2]:
USE_PSEUDOLABELS = True

## загрузка данных

In [3]:
data_train = pd.read_csv('../../data/onetwotrip_challenge_train.csv')
data_test = pd.read_csv('../../data/onetwotrip_challenge_test.csv')
print ('размеры:', data_train.shape, data_test.shape)

размеры: (196056, 43) (455011, 37)


In [4]:
if USE_PSEUDOLABELS:
    pseudolabels = pd.read_csv('../pseudolabels_692673.csv', index_col=0)['proba'].values

## посмотрим на данные

обратите внимание, как выводятся дата-фреймы

In [5]:
data_train.head()

,orderid,userid,field0,field1,field2,field3,field4,field5,field6,field7,...,indicator_goal22,indicator_goal23,indicator_goal24,indicator_goal25,goal21,goal22,goal23,goal24,goal25,goal1
0,0,10d654494cbe97bbb25d51ead2600679aff9e097924add...,0,-0.626508,11,12,1,1,0,1,...,1,0,1,1,0,1,0,0,0,0
1,1,4aafc0391f72bbcf60537aece62923baf9ce644b64ac36...,144,-0.393794,5,7,2,0,0,2,...,1,0,1,0,0,0,0,0,0,0
2,2,bac8ffef46348f587c8d17137ab01fb24aef21547c647d...,134,-0.548937,2,3,2,0,0,1,...,1,0,1,1,0,0,0,0,0,0
3,3,0392247b4b87674aba2c32bf2292b105771a6a376871be...,0,-0.238651,10,11,1,1,3,2,...,1,0,1,1,0,0,0,0,0,0
4,4,d1aeefef311bbeb4bd84876c8d49421f276674527d5578...,0,-0.704079,8,11,1,1,0,1,...,1,0,0,1,0,0,0,0,0,0


In [6]:
data_test.head()

,orderid,userid,field0,field1,field2,field3,field4,field5,field6,field7,...,field25,field26,field27,field28,field29,indicator_goal21,indicator_goal22,indicator_goal23,indicator_goal24,indicator_goal25
0,0,3a6a6af55e097d3f92705936a7ea3ca8aef651f5966832...,0,-0.548937,10,10,1,1,0,1,...,1,1,1,0,4,1,1,0,0,1
1,1,2df10f61960163da94a4294773ed9c865296e37c330304...,82,-0.626508,3,4,3,0,0,1,...,1,1,1,0,1,1,1,0,1,1
2,2,20dc3fec5b5eb42fbfe08119063c3a0010a73c7ec94abb...,0,-0.548937,6,8,1,1,0,2,...,35,1,1,0,2,1,0,0,1,1
3,3,ed75b3496977bac207eccb59dc91fe9a8d6a27777a6422...,6,0.304348,7,7,2,0,10,1,...,1,3,1,0,3,1,1,0,1,1
4,4,a346d08351c5fd0bda82984ed7c8b12b6395829da5b857...,115,-0.471365,3,3,2,0,0,1,...,1,1,1,0,1,1,1,0,1,1


### Проведем немного FE (feature enginering):

In [7]:
full_data = pd.concat([data_train, data_test], axis=0, sort=False)

In [8]:
full_data = full_data.fillna(-1)

In [9]:

# full_data.drop(['goal21'], inplace=True, axis=1)
# full_data.drop(['goal22'], inplace=True, axis=1)
# full_data.drop(['goal23'], inplace=True, axis=1)
# full_data.drop(['goal24'], inplace=True, axis=1)
# full_data.drop(['goal25'], inplace=True, axis=1)
# full_data.drop(['indicator_goal21'], inplace=True, axis=1)
# full_data.drop(['indicator_goal22'], inplace=True, axis=1)
# full_data.drop(['indicator_goal23'], inplace=True, axis=1)
# full_data.drop(['indicator_goal24'], inplace=True, axis=1)
# full_data.drop(['indicator_goal25'], inplace=True, axis=1)
first_task_labels = full_data['goal1'].copy()
full_data.drop('goal1', inplace=True, axis=1)

In [10]:
from tqdm import tqdm_notebook

In [11]:
%%time


group_by_features = ['userid']#, 'indicator_goal21', 'indicator_goal22',
                     #'indicator_goal23', 'indicator_goal24', 'indicator_goal25']
target_columns = ['goal21', 'goal22', 'goal23', 'goal24', 'goal25']
cols = list(full_data.columns.delete(1)[:-5])
print(cols)
for field in ['indicator_goal21', 'indicator_goal22', 'indicator_goal23', 'indicator_goal24', 'indicator_goal25']:
    cols.remove(field)

for group_by_feature in group_by_features:
    for fname in tqdm_notebook(cols):
        full_data[f'{fname}_to_mean_{group_by_feature}'] = full_data[fname] / full_data.groupby([group_by_feature])[
            fname].transform('mean').replace(-np.inf, np.nan).replace(np.inf, np.nan).astype(np.float32)
        full_data[f'{fname}_to_std_{group_by_feature}'] = full_data[fname] / full_data.groupby([group_by_feature])[
            fname].transform('std').replace(-np.inf, np.nan).replace(np.inf, np.nan).astype(np.float32)

['orderid', 'field0', 'field1', 'field2', 'field3', 'field4', 'field5', 'field6', 'field7', 'field8', 'field9', 'field10', 'field11', 'field12', 'field13', 'field14', 'field15', 'field16', 'field17', 'field18', 'field19', 'field20', 'field21', 'field22', 'field23', 'field24', 'field25', 'field26', 'field27', 'field28', 'field29', 'indicator_goal21', 'indicator_goal22', 'indicator_goal23', 'indicator_goal24', 'indicator_goal25']



CPU times: user 40.5 s, sys: 663 ms, total: 41.1 s
Wall time: 40.9 s


In [12]:
full_data.shape

(651067, 104)

In [13]:
full_data['tickets_amount_wout_baby'] = full_data['field15'] - full_data['field9']
full_data['order_cost_1'] = full_data['field14'] * full_data['field15']
full_data['prices_ratio'] = full_data['field1'] / (full_data['field14'] * full_data['field15'])

In [14]:
full_data = pd.merge(full_data.reset_index(), full_data.groupby('userid')['field0'].agg(['count']), on='userid', how='left').set_index('orderid')

In [15]:
full_data = pd.merge(full_data.reset_index(), full_data.groupby('userid')['field1'].agg(['mean','std']), on='userid').rename(columns={'mean':'field1_stat_mean','std':'field1_stat_std'})
full_data = pd.merge(full_data, full_data.groupby('userid')['field14'].agg(['mean','std']), on='userid').rename(columns={'mean':'field14_stat_mean','std':'field14_stat_std'})
# full_data = pd.merge(full_data, full_data.groupby('userid')['field24'].agg(['mean','std']), on='userid').rename(columns={'mean':'field24_stat_mean','std':'field24_stat_std'})
full_data = full_data.set_index('orderid')

In [16]:
data_train = full_data[full_data['goal21'] >= 0]
data_test = full_data[full_data['goal21'] < 0]

## получаем таблички для обучения

In [17]:
y = np.array(data_train[target_columns]) # целевые переменные для второй задачи задачи

In [18]:
first_task_labels

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
455006   -1.0
455007   -1.0
455008   -1.0
455009   -1.0
455010   -1.0
Name: goal1, Length: 651067, dtype: float64

In [19]:
# что получилось
data_train['goal1'] = first_task_labels.values.reshape(-1)[:len(data_train)]
data_test['goal1'] = pseudolabels
data_train.head()

,index,userid,field0,field1,field2,field3,field4,field5,field6,field7,...,field29_to_std_userid,tickets_amount_wout_baby,order_cost_1,prices_ratio,count,field1_stat_mean,field1_stat_std,field14_stat_mean,field14_stat_std,goal1
orderid,,,,,,,,,,,,,,,,,,,,,
0,0,10d654494cbe97bbb25d51ead2600679aff9e097924add...,0,-0.626508,11,12,1,1,0,1,...,NaN,1,-0.661308,0.947377,1,-0.626508,NaN,-0.661308,NaN,0.0
1,1,4aafc0391f72bbcf60537aece62923baf9ce644b64ac36...,144,-0.393794,5,7,2,0,0,2,...,1.414214,1,-0.101043,3.897296,2,-0.393794,0.000000,-0.171076,0.099042,0.0
174861,174861,4aafc0391f72bbcf60537aece62923baf9ce644b64ac36...,0,-0.393794,12,2,1,1,0,2,...,2.828427,1,-0.241109,1.633260,2,-0.393794,0.000000,-0.171076,0.099042,0.0
2,2,bac8ffef46348f587c8d17137ab01fb24aef21547c647d...,134,-0.548937,2,3,2,0,0,1,...,0.471405,1,-0.661308,0.830077,2,-0.587722,0.054851,-0.661308,0.000000,0.0
173820,173820,bac8ffef46348f587c8d17137ab01fb24aef21547c647d...,0,-0.626508,10,10,1,1,0,1,...,1.885618,1,-0.661308,0.947377,2,-0.587722,0.054851,-0.661308,0.000000,0.0


In [20]:
# удаляем ненужные признаки
train_userid = data_train['userid']
ids = data_test.pop('index') # сохраняем id для теста


In [21]:
data_test.drop(['userid'], inplace=True, axis=1)
data_test.drop(target_columns, inplace=True, axis=1)


In [22]:
cols = data_test.columns # значимые колонки
cols

Index(['field0', 'field1', 'field2', 'field3', 'field4', 'field5', 'field6',
       'field7', 'field8', 'field9',
       ...
       'field29_to_std_userid', 'tickets_amount_wout_baby', 'order_cost_1',
       'prices_ratio', 'count', 'field1_stat_mean', 'field1_stat_std',
       'field14_stat_mean', 'field14_stat_std', 'goal1'],
      dtype='object', length=106)

In [23]:
data_train = data_train[cols]

### Эксперименты

сначала делим выборку на обучение и тест

не самое лучшее решение, но для быстроты экспериментов сгодится

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_train, y[:, 0], test_size=0.3, random_state=13)

### Случайный лес


строим по одному дереву и вычисляем метрику качества (ROC AUC)

обратите внимание на прогресс-бар

In [25]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import roc_auc_score

# model = RandomForestClassifier(max_features=1, n_estimators=1, oob_score=False, warm_start=True, random_state=1)

# aucs = []
# for t in tqdm_notebook(list(range(1, 101))):
#     model.set_params(n_estimators=t)
#     model.fit(X_train, y_train)
#     a = model.predict_proba(X_test)[:, 1]
#     q = roc_auc_score(y_test, a)
#     aucs.append(q)
    
# plt.plot(range(1, 101), aucs)

In [26]:
# # аналогичные эксперименты с max_features=2

# model = RandomForestClassifier(max_features=2, n_estimators=1, oob_score=False, warm_start=True, random_state=1)

# aucs = []
# for t in tqdm_notebook(list(range(1, 101))):
#     model.set_params(n_estimators=t)
#     model.fit(X_train, y_train)
#     a = model.predict_proba(X_test)[:, 1]
#     q = roc_auc_score(y_test, a)
#     aucs.append(q)
    
# plt.plot(range(1, 101), aucs)   

### Модель с кросс-валидацией:

In [27]:
from sklearn.model_selection import GroupKFold, StratifiedKFold

N_SPLITS = 5
gr_kfold = StratifiedKFold(n_splits=N_SPLITS)

In [28]:
import lightgbm as lgb
model = lgb.LGBMClassifier(bagging_fraction=0.5539674763124854, bagging_freq=5,
               boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.7841488307689868, eval_metric='AUC',
               importance_type='split', lambda_l1=7.892207361708597,
               lambda_l2=8.705620624758234, learning_rate=0.03909410437165517,
               max_bin=270, max_depth=10, metric='AUC', min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
               n_jobs=16, num_leaves=15, objective='binary', random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, seed=42, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0,
               verbose=0)

In [29]:
from sklearn.metrics import roc_auc_score
coefs = np.array([0.28, 0.27, 0.09, 0.18, 0.18])
oof_scores = np.zeros(len(target_columns))

In [30]:
%%time

param = {'bagging_fraction': 0.5539674763124854, 'bagging_freq':5,
               'boosting_type': 'gbdt', 'class_weight':None,
               'colsample_bytree': 0.7841488307689868, 'eval_metric': 'AUC',
               'importance_type': 'split', 'lambda_l1': 7.892207361708597,
               'lambda_l2': 8.705620624758234, 'learning_rate': 0.03909410437165517,
               'max_bin': 270, 'max_depth': 10, 'metric': 'AUC', 'min_child_samples': 20,
               'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 2000,
               'n_jobs': 16, 'num_leaves': 15, 'objective': 'binary', 'random_state': 13,
               'seed': 13, 'silent': False,
               'verbose': 100, 'early_stopping_rounds': 200}
param['metric'] = 'auc'

for i, name in enumerate(target_columns):
    oof_predictions = np.zeros(data_train.shape[0])

    for train_idxs, test_idxs in gr_kfold.split(data_train, y[:, i]):
        q = lgb.train(param, train_set=lgb.Dataset(data_train.values[train_idxs], y[train_idxs, i]), num_boost_round=500,
                      valid_sets=[lgb.Dataset(data_train.values[train_idxs], y[train_idxs, i]),
                                  lgb.Dataset(data_train.values[test_idxs], y[test_idxs, i])],
                     verbose_eval=500)
        oof_predictions[test_idxs] = q.predict(data_train.values[test_idxs])
    oof_scores[i] = roc_auc_score(y[:, i], oof_predictions[:data_train.shape[0]])
    print(f'\nOOF on {name}={oof_scores[i]}')
    print('------------------------------------------------------------------------------------\n')

Training until validation scores don't improve for 200 rounds
[500]	valid_0's auc: 0.789362	valid_1's auc: 0.746289
Early stopping, best iteration is:
[353]	valid_0's auc: 0.772373	valid_1's auc: 0.747424
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[104]	valid_0's auc: 0.736044	valid_1's auc: 0.726232
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[77]	valid_0's auc: 0.728291	valid_1's auc: 0.728137
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[102]	valid_0's auc: 0.741643	valid_1's auc: 0.701031
Training until validation scores don't improve for 200 rounds
[500]	valid_0's auc: 0.811596	valid_1's auc: 0.67642
[1000]	valid_0's auc: 0.855771	valid_1's auc: 0.678697
Early stopping, best iteration is:
[973]	valid_0's auc: 0.853776	valid_1's auc: 0.679138

OOF on goal21=0.7002498305376228
--------------------------------------------------

In [31]:
print(f'Main metric = {np.sum(oof_scores * coefs)}')
print(f'OOFs per target : {oof_scores}')

Main metric = 0.7730878816482095
OOFs per target : [0.70024983 0.69073522 0.89193277 0.78811955 0.93546639]


Main metric = 0.7729918615333461
OOFs per target : [0.70083735 0.6890152  0.89177746 0.78954658 0.93524969]

[50]	valid_0's auc: 0.709098	valid_1's auc: 0.675885
[100]	valid_0's auc: 0.726553	valid_1's auc: 0.679719
[150]	valid_0's auc: 0.736703	valid_1's auc: 0.679034
[200]	valid_0's auc: 0.747241	valid_1's auc: 0.678564
[250]	valid_0's auc: 0.756099	valid_1's auc: 0.67807
[300]	valid_0's auc: 0.762981	valid_1's auc: 0.677406
[350]	valid_0's auc: 0.771099	valid_1's auc: 0.676982
[400]	valid_0's auc: 0.778818	valid_1's auc: 0.676227
[450]	valid_0's auc: 0.783759	valid_1's auc: 0.675318
[500]	valid_0's auc: 0.789803	valid_1's auc: 0.67336
CPU times: user 2min 20s, sys: 3.32 s, total: 2min 23s

Воспользуемся bayes_opt для подбора параметров:

In [32]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score

coefs = np.array([0.28, 0.27, 0.09, 0.18, 0.18])
oof_scores = np.zeros(len(target_columns))

def lgb_bayesian(max_depth,
                 lambda_l1,
                 lambda_l2,
                 bagging_fraction,
                 bagging_freq,
                 colsample_bytree,
                 learning_rate,
                 num_leaves,
                 max_bin
                      ):
    params = {
        'boosting_type': 'gbdt',
        'metric': 'AUC',
        'objective': 'binary',
        'eval_metric': 'AUC',
        'n_jobs': 16,
        'seed': 42,
        'early_stopping_rounds': 150,
        'n_estimators': 2000,
        'learning_rate': learning_rate,
        'max_depth': int(max_depth),
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': int(bagging_freq),
        'colsample_bytree': colsample_bytree,
        'verbose': 0,
        'early_stopping_rounds': 200
    }
    for i, name in enumerate(target_columns):
        oof_predictions = np.zeros(data_train.shape[0])
        for train_idxs, test_idxs in gr_kfold.split(data_train, y[:, i]):
            model = lgb.LGBMClassifier(boosting_type=params['boosting_type'],
                                        metric='AUC',
                                        objective='binary',
                                        eval_metric='AUC',
                                        n_jobs= 16,
                                        seed=13,
        #                                 early_stopping_rounds=150,
                                        n_estimators= 2000,
                                        learning_rate=learning_rate,
                                        max_depth=int(max_depth),
                                        lambda_l1=lambda_l1,
                                        lambda_l2=lambda_l2,
                                        bagging_fraction=bagging_fraction,
                                        bagging_freq=int(bagging_freq),
                                        max_bin=int(max_bin),
                                        num_leaves=int(num_leaves),
                                        colsample_bytree=colsample_bytree,
                                        verbose=0
        #                                 valid_sets=[lgb.Dataset(X_train, y_train), lgb.Dataset(X_test, y_test)]
                                        )
            model.fit(data_train.values[train_idxs],
                      y[train_idxs, i],
                      early_stopping_rounds=200,
                      eval_metric='AUC',
                      eval_set=(data_train.values[test_idxs], y[test_idxs, i]), verbose=0)
            oof_predictions[test_idxs] = model.predict_proba(data_train.values[test_idxs])[:, 1]
        oof_scores[i] = roc_auc_score(y[:, i], oof_predictions[:data_train.shape[0]])
        print(f'\nOOF on {name}={oof_scores[i]}')
        print('------------------------------------------------------------------------------------\n')
    return np.sum(oof_scores * coefs)

In [33]:
bounds_LGB = {
    'max_depth': (2, 25),
    'lambda_l1': (0, 10),
    'lambda_l2': (0, 10),
    'bagging_fraction': (0.4, 0.9),
    'bagging_freq': (1, 10),
    'colsample_bytree': (0.4, 0.9),
    'learning_rate': (0.025, 0.1),
    'num_leaves': (2, 20),
    'max_bin': (2, 1000)
}


lgbm_BO = BayesianOptimization(lgb_bayesian, bounds_LGB, random_state=13 * 2)

In [34]:
%%time

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    lgbm_BO.maximize(init_points=10, n_iter=10, xi=0, alpha=1e-3)

|   iter    |  target   | baggin... | baggin... | colsam... | lambda_l1 | lambda_l2 | learni... |  max_bin  | max_depth | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------

OOF on goal21=0.7007246532898833
------------------------------------------------------------------------------------


OOF on goal22=0.6903165344655299
------------------------------------------------------------------------------------


OOF on goal23=0.8922563838237593
------------------------------------------------------------------------------------


OOF on goal24=0.7886138201629754
------------------------------------------------------------------------------------


OOF on goal25=0.9353622172530902
------------------------------------------------------------------------------------

|  1        |  0.7732   |  0.554    |  5.675    |  0.7841   |  7.892    |  8.706    |  0.03909  |  271.0    |  13.41    |  15.3     


OOF on goal21=0.6999914234295941
------------------------------------------------------------------------------------


OOF on goal22=0.6964295550553457
------------------------------------------------------------------------------------


OOF on goal23=0.8893726933856534
------------------------------------------------------------------------------------


OOF on goal24=0.7898649757438886
------------------------------------------------------------------------------------


OOF on goal25=0.9333877346333262
------------------------------------------------------------------------------------

|  11       |  0.7743   |  0.411    |  2.575    |  0.6932   |  0.7595   |  0.7011   |  0.09531  |  999.9    |  2.284    |  18.83    |

OOF on goal21=0.6977951958118761
------------------------------------------------------------------------------------


OOF on goal22=0.6937291534367409
------------------------------------------------------------------------------------


OOF on goal23=0.889804318

OOF on goal21=0.702059666055665
------------------------------------------------------------------------------------


OOF on goal22=0.6980528403079374
------------------------------------------------------------------------------------


OOF on goal23=0.8930573167060869
------------------------------------------------------------------------------------


OOF on goal24=0.7902882834027807
------------------------------------------------------------------------------------


OOF on goal25=0.9372977864812237
------------------------------------------------------------------------------------

## 0.7764

In [35]:
# model_params = {
#     'max_depth': int(lgbm_BO.max['params']['max_depth']),
# #     'n_estimators': int(lgbm_BO.max['params']['n_estimators']),
#     'learning_rate': lgbm_BO.max['params']['learning_rate'],
#     'num_leaves': int(lgbm_BO.max['params']['num_leaves']),
#     'max_bin': int(lgbm_BO.max['params']['max_bin']),
#     'boosting_type': 'gbdt',
#     'metric': 'AUC',
#     'objective': 'binary',
#     'eval_metric': 'AUC',
#     'n_jobs': 16,
#     'seed': 42,
# #     'early_stopping_rounds': 150,
#     'n_estimators': 2000,
#     'lambda_l1': lambda_l1,
#     'lambda_l2': lambda_l2,
#     'bagging_fraction': lgbm_BO.max['params']['bagging_fraction'],
#     'bagging_freq': int(lgbm_BO.max['params']['bagging_freq']),
#     'colsample_bytree': lgbm_BO.max['params']['colsampel_bytree'],
#     'verbose': 0
# }

In [36]:
from tqdm import tqdm_notebook

In [37]:
coefs = np.array([0.28, 0.27, 0.09, 0.18, 0.18])
oof_scores = np.zeros(len(target_columns))

a = np.zeros((data_test.shape[0], len(target_columns)))

for i, name in enumerate(target_columns):
    oof_predictions = np.zeros(data_train.shape[0])
    for train_idxs, test_idxs in tqdm_notebook(gr_kfold.split(data_train, y[:, i])):
        model = lgb.LGBMClassifier(
            max_depth=int(lgbm_BO.max['params']['max_depth']),
            learning_rate=lgbm_BO.max['params']['learning_rate'],
            num_leaves=int(lgbm_BO.max['params']['num_leaves']),
            max_bin=int(lgbm_BO.max['params']['max_bin']),
            boosting_type='gbdt',
            metric='AUC',
            objective='binary',
            eval_metric='AUC',
            n_jobs=16,
            seed=42,
        #     'early_stopping_rounds': 150,
            n_estimators=2000,
            lambda_l1=lgbm_BO.max['params']['lambda_l1'],
            lambda_l2=lgbm_BO.max['params']['lambda_l2'],
            bagging_fraction=lgbm_BO.max['params']['bagging_fraction'],
            bagging_freq=int(lgbm_BO.max['params']['bagging_freq']),
            colsample_bytree=lgbm_BO.max['params']['colsample_bytree'],
            verbose=0
        )
        model.fit(data_train.values[train_idxs],
                  y[train_idxs, i],
                  early_stopping_rounds=200,
                  eval_metric='AUC',
                  eval_set=(data_train.values[test_idxs], y[test_idxs, i]), verbose=0)
        oof_predictions[test_idxs] = model.predict_proba(data_train.values[test_idxs])[:, 1]
        a[:, i] += model.predict_proba(data_test.values)[:, 1] / N_SPLITS
    oof_scores[i] = roc_auc_score(y[:, i], oof_predictions[:data_train.shape[0]])
    print(f'OOF on {name} = {roc_auc_score(y[:, i], oof_predictions)}')
print(f'Main metric: {(oof_scores * coefs).sum()}')


OOF on goal21 = 0.702502197181676



OOF on goal22 = 0.692451755055854



OOF on goal23 = 0.8926073954382717



OOF on goal24 = 0.7911376844597058



OOF on goal25 = 0.9373477668718767
Main metric: 0.7751246359050792


In [38]:
print(f'Main metric: {(oof_scores * coefs).sum()}')
print(f'OOF per targets: {oof_scores}')

Main metric: 0.7751246359050792
OOF per targets: [0.7025022  0.69245176 0.8926074  0.79113768 0.93734777]


Main metric: 0.7764283131266208
OOF per targets: [0.70349396 0.69554358 0.89260596 0.79198469 0.93756364]

In [39]:
indicator_columns = ['indicator_goal' + numb[-2:] for numb in target_columns]
for i, ind_col in enumerate(indicator_columns):
    a[:, i] = a[:, i] * data_test[ind_col]

In [40]:
pd.DataFrame(a,  columns=target_columns, index=ids.values).to_csv('lgbm.csv')

In [ ]:
%%time

from xgboost import XGBClassifier

In [ ]:
coefs = np.array([0.28, 0.27, 0.09, 0.18, 0.18])
oof_scores = np.zeros(len(target_columns))

a = np.zeros((data_test.shape[0], len(target_columns)))

for i, name in enumerate(target_columns):
    oof_predictions = np.zeros(data_train.shape[0])
    for train_idxs, test_idxs in tqdm_notebook(gr_kfold.split(data_train, y[:, i])):
        model = XGBClassifier(n_estimators=100, gpu_hist=True, nthread=16, objective='binary:logistic',
                              max_depth=4,  max_leaves=50, colsample_bytree=0.8,
                              eta=0.3, min_child_weight=1, subsample=1, eval_metric='auc',
                              alpha=0.15, reg_lambda=0.85)
        model.fit(data_train.values[train_idxs],
                  y[train_idxs, i], 
                 )
        oof_predictions[test_idxs] = model.predict_proba(data_train.values[test_idxs])[:, 1]
        a[:, i] += model.predict_proba(data_test.values)[:, 1] / N_SPLITS
    oof_scores[i] = roc_auc_score(y[:, i], oof_predictions[:data_train.shape[0]])
    print(f'OOF on {name} = {roc_auc_score(y[:, i], oof_predictions)}')
print(f'Main metric: {(oof_scores * coefs).sum()}')

In [ ]:
print(f'Main metric: {(oof_scores * coefs).sum()}')
print(f'OOF per targets: {oof_scores}')

Main metric: 0.7678450795841563
OOF per targets: [0.69211167 0.68293963 0.88981633 0.78325469 0.93661555]

In [ ]:
indicator_columns = ['indicator_goal' + numb[-2:] for numb in target_columns]
for i, ind_col in enumerate(indicator_columns):
    a[:, i] = a[:, i] * data_test[ind_col]

In [ ]:
pd.DataFrame(a,  columns=target_columns, index=ids.values).to_csv('xgb.csv')

In [ ]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score

coefs = np.array([0.28, 0.27, 0.09, 0.18, 0.18])
oof_scores = np.zeros(len(target_columns))

def lgb_bayesian(max_depth,
                 lambda_l1,
                 lambda_l2,
                 bagging_fraction,
                 bagging_freq,
                 colsample_bytree,
                 learning_rate,
                 num_leaves,
                 max_bin
                      ):
    params = {
        'boosting_type': 'gbdt',
        'metric': 'AUC',
        'objective': 'binary',
        'eval_metric': 'AUC',
        'n_jobs': 16,
        'seed': 42,
        'early_stopping_rounds': 150,
        'n_estimators': 2000,
        'learning_rate': learning_rate,
        'max_depth': int(max_depth),
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': int(bagging_freq),
        'colsample_bytree': colsample_bytree,
        'verbose': 0,
        'early_stopping_rounds': 200
    }
    for i, name in enumerate(target_columns):
        oof_predictions = np.zeros(data_train.shape[0])
        for train_idxs, test_idxs in gr_kfold.split(data_train, y[:, i]):
    model = CatBoostClassifier(max_depth=max_depth, n_estimators=n_estimators,
                               learning_rate=lr, objective='CrossEntropy', eval_metric='AUC',
                               task_type='GPU', verbose=0, thread_count=16)
    model.fit(X_train, y_train)

    return roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

In [ ]:
bounds_cat = {
    'max_depth': (1, 16),
    'n_estimators': (1, 1000),
    'lr': (1e-3, 1),
}

cat_BO = BayesianOptimization(catboost_bayesian, bounds_cat, random_state=13 * 2)

In [ ]:
%%time
import warnings


with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    cat_BO.maximize(init_points=20, n_iter=20, acq='ucb')

In [ ]:
model = CatBoostClassifier(max_depth=int(cat_BO.max['params']['max_depth']),
                           n_estimators=int(cat_BO.max['params']['n_estimators']),
                           learning_rate=cat_BO.max['params']['lr'],
                           objective='CrossEntropy', eval_metric='AUC',
                               task_type='GPU', verbose=0, thread_count=16)

In [ ]:
model.fit(data_train, y)

In [ ]:
a = model.predict_proba(data_test)[:, 1] # вероятности за 1й класс

In [ ]:
pd.DataFrame(a,  columns=['proba'], index=ids.values).to_csv('ctb.csv')

### Обучение и формирование ответа

In [ ]:
model.fit(data_train, y)

In [ ]:
a = model.predict_proba(data_test)[:, 1] # вероятности за 1й класс

In [ ]:
pd.DataFrame(a,  columns=['proba'], index=ids.values).to_csv('dj1_01_.csv')